In [1]:

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
# Set the resolution of the plotted figures
plt.rcParams['figure.dpi'] = 200

# Configure Seaborn plot styles: Set background color and use dark grid
sns.set(rc={'axes.facecolor': '#faded9'}, style='darkgrid')

In [1]:
ls

 Volume in drive C has no label.
 Volume Serial Number is E441-4DE6

 Directory of C:\Users\HP\Desktop\MLpro\health_notebook

02/13/2024  09:53 PM    <DIR>          .
02/13/2024  08:13 PM    <DIR>          ..
02/13/2024  09:53 PM    <DIR>          .ipynb_checkpoints
02/13/2024  09:52 PM    <DIR>          data
02/13/2024  09:47 PM             4,901 Heart_Disease_Prediction.ipynb
02/13/2024  08:46 PM               533 Readme
               2 File(s)          5,434 bytes
               4 Dir(s)  144,189,079,552 bytes free


In [3]:
# Read dataset
df = pd.read_csv('./data/heart.csv')
df 

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [6]:
with open('Data Descriptions.txt', 'r') as file:
    # Read the contents of the file
    data = file.read()
    
data

'Dataset Description:\nVariable\tDescription\nage\tAge of the patient in years\nsex\tGender of the patient (0 = male, 1 = female)\ncp\tChest pain type:\n0: Typical angina\n1: Atypical angina\n2: Non-anginal pain\n3: Asymptomatic\ntrestbps\tResting blood pressure in mm Hg\nchol\tSerum cholesterol in mg/dl\nfbs\tFasting blood sugar level, categorized as above 120 mg/dl (1 = true, 0 = false)\nrestecg\tResting electrocardiographic results:\n0: Normal\n1: Having ST-T wave abnormality\n2: Showing probable or definite left ventricular hypertrophy\nthalach\tMaximum heart rate achieved during a stress test\nexang\tExercise-induced angina (1 = yes, 0 = no)\noldpeak\tST depression induced by exercise relative to rest\nslope\tSlope of the peak exercise ST segment:\n0: Upsloping\n1: Flat\n2: Downsloping\nca\tNumber of major vessels (0-4) colored by fluoroscopy\nthal\tThalium stress test result:\n0: Normal\n1: Fixed defect\n2: Reversible defect\n3: Not described\ntarget\tHeart disease status (0 = no

In [4]:
# Define the continuous features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Identify the features to be converted to object data type
features_to_convert = [feature for feature in df.columns if feature not in continuous_features]

# Convert the identified features to object data type
df[features_to_convert] = df[features_to_convert].astype('object')

df.dtypes

age           int64
sex          object
cp           object
trestbps      int64
chol          int64
fbs          object
restecg      object
thalach       int64
exang        object
oldpeak     float64
slope        object
ca           object
thal         object
target       object
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    object 
 2   cp        303 non-null    object 
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    object 
 6   restecg   303 non-null    object 
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    object 
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    object 
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  target    303 non-null    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 33.3+ KB


#### Inferences:
- Number of Entries: Total of 303 entries, ranging from index 0 to 302.
- Columns: 14 columns in the dataset corresponding to various attributes of the patients and results of tests.
- Data Types: Most of the columns (13 out of 14) are of the int64 data type. Only the oldpeak column is of the float64 data type.
- Missing Values: No

#### Note: 
From the above we can see that, we have 9 columns (sex, cp, fbs, restecg, exang, slope, ca, thal, and target) are indeed numerical in terms of data type, but categorical in terms of their semantics. 
- These features should be converted to string (object) data type for proper analysis and interpretation:

In [8]:
# Define the continuous features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Identify the features to be converted to object data type
features_to_convert = [feature for feature in df.columns if feature not in continuous_features]

# Convert the identified features to object data type
df[features_to_convert] = df[features_to_convert].astype('object')

df.dtypes

age           int64
sex          object
cp           object
trestbps      int64
chol          int64
fbs          object
restecg      object
thalach       int64
exang        object
oldpeak     float64
slope        object
ca           object
thal         object
target       object
dtype: object

In [9]:
df.describe()

,age,trestbps,chol,thalach,oldpeak
count,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,131.623762,246.264026,149.646865,1.039604
std,9.082101,17.538143,51.830751,22.905161,1.161075
min,29.000000,94.000000,126.000000,71.000000,0.000000
25%,47.500000,120.000000,211.000000,133.500000,0.000000
50%,55.000000,130.000000,240.000000,153.000000,0.800000
75%,61.000000,140.000000,274.500000,166.000000,1.600000
max,77.000000,200.000000,564.000000,202.000000,6.200000


In [10]:
# Get the summary statistics for categorical variables
df.describe(include='object')

,sex,cp,fbs,restecg,exang,slope,ca,thal,target
count,303,303,303,303,303,303,303,303,303
unique,2,4,2,3,2,3,5,4,2
top,1,0,0,1,0,2,0,2,1
freq,207,143,258,152,204,142,175,166,165
